In [ ]:
import argparse
import os
import time
from pprint import pprint
import googleapiclient.discovery
from six.moves import input

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/home/sathish/gcp_pem.json'

data_node_instances={'datanode1':{'instance_name':'datanode1','hostname':'datanode1.tanu.com'},
                   'datanode2':{'instance_name':'datanode2','hostname':'datanode2.tanu.com'}}

master_node_instances={'masternode1':{'instance_name':'masternode','hostname':'master.tanu.com'}}

def generate_data_node_startup_scripts(master_node_name,master_node_ip):
    script = open('new-client-startup-script.sh', 'w') 
    print('''
    #!/bin/bash

    WORK_DIR="/var/tmp/startup_dir"
    
    mkdir $WORK_DIR
    cd $WORK_DIR
    hostname=$(hostname -f)
    echo "{master_node_ip} {master_node_name}" >>/etc/hosts
    yum -y install git-core net-tools krb5-workstation
    git clone https://github.com/skumarx87/openldap_MIT-Kerberos_installation.git
    cd openldap_MIT-Kerberos_installation
    sed -i "s/idm.tanu.com/{master_node_name}/g" kerberos_ldap_installation.sh
    ./kerberos_ldap_installation.sh client_setup
    '''.format(master_node_ip=master_node_ip,master_node_name=master_node_name),file=script)
    script.close()

def generate_master_node_startup_scripts(master_node_name):
    script = open('new-server-startup-script.sh', 'w') 
    print('''
    #!/bin/bash
    WORK_DIR="/var/tmp/startup_dir"
    mkdir $WORK_DIR
    cd $WORK_DIR
    echo "this is startup script"
    yum -y install git-core net-tools krb5-workstation
    git clone https://github.com/skumarx87/openldap_MIT-Kerberos_installation.git
    cd openldap_MIT-Kerberos_installation
    sed -i "s/idm.tanu.com/{master_node_name}/g" kerberos_ldap_installation.sh
    ./kerberos_ldap_installation.sh server_setup
    ./kerberos_ldap_installation.sh setup_webserver
    ./kerberos_ldap_installation.sh client_setup
    ./kerberos_ldap_installation.sh create_hadoop_users

    '''.format(master_node_name=master_node_name),file=script)
    script.close()
    
def list_instances(compute, project, zone):
        result = compute.instances().list(project=project, zone=zone).execute()
        return result['items'] if 'items' in result else None
        # [END list_instances]

def list_images(compute,project,zone):
        request = compute.images().list(project=project).execute()
        print(request)
        #while request is not None:
        #       response = request.execute()
        #       print(response)
                #for image in response['items']:
                #       pprint(image)
        #       request = compute.images().list_next(previous_request=request, previous_response=response)
def list_disks(compute,project,zone):
        request = compute.disks().list(project=project, zone=zone)
        while request is not None:
            response = request.execute()
            for disk in response['items']:
                pprint(disk)
            request = service.disks().list_next(previous_request=request, previous_response=response)
        
        
def get_instance_ip(compute,project,zone,instance):
        request = compute.instances().get(project=project, zone=zone, instance=instance)
        response = request.execute()
        pprint(response)
        host_ip=response['networkInterfaces'][0]['networkIP']
        return host_ip
        
def create_instance(compute,project,zone,name,hostname,startup_script):
        image_response = compute.images().getFromFamily(project='centos-cloud', family='centos-7').execute()
        source_disk_image = image_response['selfLink']
        machine_type = "zones/%s/machineTypes/n1-standard-1" % zone
        startup_script = open(
                os.path.join(
                        os.path.abspath(''), startup_script), 'r').read()
        config = {
            'name': name,
            'machineType': machine_type,
            'hostname':hostname,
            'disks': [
                {
                    'boot': True,
                    'autoDelete': True,
                    'initializeParams': {
                        'sourceImage': source_disk_image,
                        'diskSizeGb': 10,
                    }
                }
            ],
            # Specify a network interface with NAT to access the public
            # internet.
            'networkInterfaces': [{
                'network': 'global/networks/default',
                'accessConfigs': [
                    {'type': 'ONE_TO_ONE_NAT', 'name': 'External NAT'}
                ]
            }],
            'metadata': {
            'items': [{
                # Startup script is automatically executed by the
                # instance upon startup.
                'key': 'startup-script',
                'value': startup_script
            }]
            }
        }
        return compute.instances().insert(project=project,zone=zone,body=config).execute()
            
def wait_for_operation(compute, project, zone, operation):
    print('Waiting for operation to finish...')
    while True:
        result = compute.zoneOperations().get(
            project=project,
            zone=zone,
            operation=operation).execute()

        if result['status'] == 'DONE':
            print("done.")
            if 'error' in result:
                raise Exception(result['error'])
            return result

        time.sleep(1)
        
def main(project,zone,wait=True):
        compute = googleapiclient.discovery.build('compute', 'v1')
        print("Creating Master node")
                     
        for masternode in master_node_instances:
            inst_name=master_node_instances[masternode]['instance_name']
            host_name=master_node_instances[masternode]['hostname']
            generate_master_node_startup_scripts(host_name)
            operation=create_instance(compute,project,zone,inst_name,host_name,'new-server-startup-script.sh')
            print(operation)
            wait_for_operation(compute, project, zone, operation['name'])

        master_node_inst_name=master_node_instances['masternode1']['instance_name']
        master_node_host_name=master_node_instances['masternode1']['hostname']
        master_node_ip=get_instance_ip(compute,project,zone,master_node_inst_name)
        time.sleep(80) ## wait for master node startup script completion
        for datanode in data_node_instances:
            print("Creating Data Node : {}".format(datanode))
            inst_name=data_node_instances[datanode]['instance_name']
            host_name=data_node_instances[datanode]['hostname']
            generate_data_node_startup_scripts(master_node_host_name,master_node_ip)
            create_instance(compute,project,zone,inst_name,host_name,'new-client-startup-script.sh')
            
        #instances = list_instances(compute, project, zone)
        #print('Instances in project %s and zone %s:' % (project, zone))
        #for instance in instances:
        #        print(' - ' + instance['name'])

        #list_images(compute,project,zone)
        
        #create_instance(compute,project,zone,instance_name,'idm.tanu.com')
        #get_instance_ip(compute,project,zone,instance_name)
        #list_disks(compute,project,zone)
        #generate_data_node_startup_scripts('idm.tanu.com','10.0.0.1')
        #generate_master_node_startup_scripts()


if __name__ == '__main__':
        '''
        parser = argparse.ArgumentParser(
        description=__doc__,
        formatter_class=argparse.RawDescriptionHelpFormatter)
        parser.add_argument('project_id', help='Your Google Cloud project ID.')
        parser.add_argument(
                '--zone',
                default='us-central1-f',
                help='Compute Engine zone to deploy to.')
        parser.add_argument(
                '--name', default='demo-instance', help='New instance name.')

        args = parser.parse_args()
        '''
        main('ferrous-weaver-249914','us-east1-b')
        #main(args.project_id,args.zone,args.name)


In [ ]:
from pprint import pprint

from googleapiclient import discovery
from oauth2client.client import GoogleCredentials

credentials = GoogleCredentials.get_application_default()
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/home/sathish/gcp_pem.json'
service = discovery.build('compute', 'v1', credentials=credentials)

# Project ID for this request.
project = 'ferrous-weaver-249914'  # TODO: Update placeholder value.

#request = service.images().list(project=project)
#print(request)
'''
while request is not None:
    response = request.execute()

    for image in response['items']:
        # TODO: Change code below to process each `image` resource:
        pprint(image)

    request = service.images().list_next(previous_request=request, previous_response=response)
# TODO: Change code below to process the `response` dict:
#pprint(response)
'''